In [2]:
fege

NameError: name 'fege' is not defined

In [1]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [2]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [4]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [5]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_enviar():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE enviado = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [6]:
convertir_docx_to_pdf()

  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.docx
✅ Convertido: 13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.docx
🗑️ Eliminado original: 13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.docx
✅ Convertido: 15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.docx
🗑️ Eliminado original: 15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.docx


  0%|          | 0/1 [00:00<?, ?it/s]

...

C:\Users\Usuario\Desktop\PROYECTOS\_informes\633_SLCB_26-09-2025_AGROP. VICENTE ROCA GIL SRL_CB-2025-100.docx
✅ Convertido: 633_SLCB_26-09-2025_AGROP. VICENTE ROCA GIL SRL_CB-2025-100.docx
🗑️ Eliminado original: 633_SLCB_26-09-2025_AGROP. VICENTE ROCA GIL SRL_CB-2025-100.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\7754_SLCB_25-09-2025_LEON MONDAQUE ROLY_CB-2025-112.docx
✅ Convertido: 7754_SLCB_25-09-2025_LEON MONDAQUE ROLY_CB-2025-112.docx
🗑️ Eliminado original: 7754_SLCB_25-09-2025_LEON MONDAQUE ROLY_CB-2025-112.docx


  0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Usuario\Desktop\PROYECTOS\_informes\8894_SLCB_25-09-2025_MORA MIRANDA MARCELO_CB-2025-105.docx
✅ Convertido: 8894_SLCB_25-09-2025_MORA MIRANDA MARCELO_CB-2025-105.docx
🗑️ Eliminado original: 8894_SLCB_25-09-2025_MORA MIRANDA MARCELO_CB-2025-105.docx


In [7]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [8]:
contenido = os.listdir(directorio_destino)
print(len(contenido))
contenido

14


['13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.pdf',
 '15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.pdf',
 '18062_SLCB_25-09-2025_MERCADO MERCADO RUBEN DARIO_CB-2025-104.pdf',
 '2351_SLCB_25-09-2025_CANDIA RUIZ ADALBERTO_CB-2025-102.pdf',
 '2565_SLCB_25-09-2025_CANDIA RUIZ JOSE LUIS_CB-2025-106.pdf',
 '40250_SLCB_26-09-2025_AMURRIO MELGAROJO BISMARK ELISEO_CB-2025-110.pdf',
 '41962_SLCB_25-09-2025_CANDIA ROJAS JORGE LUIS_CB-2025-108.pdf',
 '41980_SLCB_26-09-2025_CASANOVA VELASCO JULIO CESAR_CB-2025-107.pdf',
 '42229_SLCB_25-09-2025_CANDIA ROJAS JOSE LUIS_CB-2025-109.pdf',
 '42347_SLCB_25-09-2025_MERCADO KATERINE JUSTINIANO DE_CB-2025-103.pdf',
 '42378_SLCB_25-09-2025_CANDIA RUIZ ORLANDO_CB-2025-111.pdf',
 '633_SLCB_26-09-2025_AGROP. VICENTE ROCA GIL SRL_CB-2025-100.pdf',
 '7754_SLCB_25-09-2025_LEON MONDAQUE ROLY_CB-2025-112.pdf',
 '8894_SLCB_25-09-2025_MORA MIRANDA MARCELO_CB-2025-105.pdf']

In [9]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

CB-2025-99
[INFO] El archivo '13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '13187_SLCB_25-09-2025_SUAREZ CUELLAR REMBERTO_CB-2025-99.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-99
✅ Archivo eliminado.
CB-2025-101
[INFO] El archivo '15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '15162_SLCB_26-09-2025_VELASCO ANEZ MARCO ANTONIO_CB-2025-101.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-101
✅ Archivo eliminado.
CB-2025-104
[INFO] El archivo '18062_SLCB_25-09-2025_MERCADO MERCADO RUBEN DARIO_CB-2025-104.pdf' no existe. Se creará uno nuevo.
[OK] Archivo '18062_SLCB_25-09-2025_MERCADO MERCADO RUBEN DARIO_CB-2025-104.pdf' subido exitosamente.
✔️ Se registro link para CITE: CB-2025-104
✅ Archivo eliminado.
CB-2025-102
[INFO] El archivo '2351_SLCB_25-09-2025_CANDIA RUIZ ADALBERTO_CB-2025-102.pdf' no existe. Se creará uno nuevo.
[OK

In [10]:
import psycopg2

In [11]:
def obtener_conexion():
    return psycopg2.connect(
        host='localhost', 
        port=5433, 
        database='utea', 
        user='postgres', 
        password='A123456*')
    
def insertar_mensaje_whatsapp(cod_canero, nombre_canero, numero_contac, mensaje, enviado=False, fecha_enviado=None):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            INSERT INTO notificaciones_wsp.msj_whatsapp (
                cod_canero, nombre_canero, numero_cel, mensaje, enviado, fecha_enviado, numero_contac
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (cod_canero, nombre_canero, 0, mensaje, enviado, fecha_enviado, numero_contac))
        
        conexion.commit()
        cursor.close()
        print("Mensaje insertado correctamente.")
    except Exception as e:
        conexion.rollback()
        print("Error inesperado")
    return None

def marcar_resumen_lib_enviado(id_resumen):
    try:
        conexion = obtener_conexion()
        cursor = conexion.cursor()
        cursor.execute("""
            UPDATE drones_control_bio.resumen_control_bio
            SET enviado = true
            WHERE id = %s
        """, (id_resumen,))
        conexion.commit()
        cursor.close()
        print(f"Mensaje con id={id_resumen} marcado como enviado.")
    except Exception as e:
        conexion.rollback()
        print(f"Error inesperado: {e}")
    finally:
        if conexion:
            conexion.close()
    return None

In [12]:
PATH_XLSX_CONTAC=r'G:\\Ingenio Azucarero Guabira S.A\\UTEA - SEMANAL - EQUIPO AVIACION UTEA\\Pulverizacion\\Data\\CONTACTOS.xlsx'

In [19]:
sin_enviar = get_registros_sin_enviar()

In [20]:
cods_ca = list(set(sin_enviar['id']))
cods_ca

[97, 99, 105, 106, 109, 111, 113, 116, 118, 89]

In [21]:
df_contac = pd.read_excel(PATH_XLSX_CONTAC)
df_contac.fillna(0, inplace=True)
df_contac = df_contac[['cod_ca', 'nom_ca', 'telf01', 'telf02', 'telf03']]

In [22]:
df_contac.head(3)

,cod_ca,nom_ca,telf01,telf02,telf03
0,0,CAÑERO TEST,78194371,0.0,0.0
1,86,AGUILERA TARADELLES JOSE LUIS,67759316,77010520.0,68848905.0
2,180,ELISEO AMURRIO,67559958,0.0,0.0


In [23]:
sin_enviar.head(3)

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca,enviado
0,99,2025-09-24,66,5448,HUANCA MENCHAQUE ROLY,5.67,CB-2025-92,545,TAJIBO--ANZE CUAJIRA,['TRICHOGRAMMA'],https://drive.google.com/file/d/1yMuoCSTlCPM3X...,True,False
1,89,2025-09-18,50,3412,ESPINOZA VILLCA DIONICIO,22.30,CB-2025-82,1561,OKINAWA I--DIONISIO ESPINOZA,['TRICHOGRAMMA'],https://drive.google.com/file/d/1gDmLPzM6rW9Ua...,True,False
2,97,2025-09-24,8,1165,BEJARANO GANDARILLA ERWIN,11.03,CB-2025-90,40,SAN VICENTE--ERWIN BEJARANO,['TRICHOGRAMMA'],https://drive.google.com/file/d/1g8YpDinDp3gkD...,True,False


In [24]:
for index, row in sin_enviar.iterrows():    
    contac_filtro = df_contac[df_contac['cod_ca']==row['cod_canero']]
    if not contac_filtro.empty:  # Verifica si hay al menos una fila
        contac = contac_filtro.iloc[0]
        telefonos = [int(contac['telf01']), int(contac['telf02']), int(contac['telf03'])]
        telefonos_validos = [telefono for telefono in telefonos if len(str(telefono)) == 8]
        if (len(telefonos_validos) == 0):
            print(f'Codigo: {cod} no tinene numeros validos registrados')
            continue
        
        for num in telefonos_validos:
            msj = f"Saludos *{row['nombre_canero']}*, le escribimos desde *UTEA - GUABIRA* para hacer llegar una copia del informe de servicio de control biologico en caña de azucar. Adjunto informe: {row['link']}"
            insertar_mensaje_whatsapp(
                cod_canero=row['cod_canero'],
                nombre_canero=row['nombre_canero'],
                numero_contac=f'591{num}@s.whatsapp.net',
                mensaje=msj
            )
            marcar_resumen_lib_enviado(row['id'])
    else:
        print(f"Codigo: {row['cod_canero']} - {row['nombre_canero']} no tiene registro telefonico")

Codigo: 5448 - HUANCA MENCHAQUE ROLY no tiene registro telefonico
Codigo: 3412 - ESPINOZA VILLCA DIONICIO no tiene registro telefonico
Codigo: 1165 - BEJARANO GANDARILLA ERWIN no tiene registro telefonico
Codigo: 16213 - ZAPATA SERON SIMON no tiene registro telefonico
Codigo: 18062 - MERCADO MERCADO RUBEN DARIO no tiene registro telefonico
Codigo: 2351 - CANDIA RUIZ ADALBERTO no tiene registro telefonico
Codigo: 13187 - SUAREZ CUELLAR REMBERTO no tiene registro telefonico
Codigo: 2565 - CANDIA RUIZ JOSE LUIS no tiene registro telefonico
Codigo: 42229 - CANDIA ROJAS JOSE LUIS no tiene registro telefonico
Codigo: 42378 - CANDIA RUIZ ORLANDO no tiene registro telefonico
